<a href="https://colab.research.google.com/github/ykitaguchi77/CongenitalGlaucoma_AI_project/blob/main/DataSplit(stratified_one_subject_leave_out)1.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Data_split for one-subject-leave-out stratified 5-fold crossvalidation**

In [ ]:
"""
Leave one subject out cross validation + 5-fold stratified cross validation

・1症例を抜き出し、その症例のすべての画像をテスト画像とする
・残りの症例の内斜視、外斜視、斜視なし群を、同じ症例が群をまたがないように5分割する。
・5分割したデータセットのうち4つをtraining、1つをvalidationとして用いてトレーニングを行い、抜き出した1症例のそれぞれの画像のおける正解率を算出する。これを5回繰り返してcross validationとする。

"""

In [15]:
def make_path_list(dir):
    path_list =  [file for file in glob.glob(dir+"/*") if os.path.isfile(file) == True ]
    return path_list

gla_ortho_path = r"F:\先天性緑内障\データ引継ぎ\children_d"
pathlist = make_path_list(gla_ortho_path)
print(pathlist)



['F:\\先天性緑内障\\データ引継ぎ\\children_d\\1546_2.jpg', 'F:\\先天性緑内障\\データ引継ぎ\\children_d\\1546_6.jpg', 'F:\\先天性緑内障\\データ引継ぎ\\children_d\\1546_7.jpg', 'F:\\先天性緑内障\\データ引継ぎ\\children_d\\1546_9.jpg', 'F:\\先天性緑内障\\データ引継ぎ\\children_d\\1962_1.jpg', 'F:\\先天性緑内障\\データ引継ぎ\\children_d\\2004_12.jpg', 'F:\\先天性緑内障\\データ引継ぎ\\children_d\\2004_28.jpg', 'F:\\先天性緑内障\\データ引継ぎ\\children_d\\2004_33.jpg', 'F:\\先天性緑内障\\データ引継ぎ\\children_d\\2004_34.jpg', 'F:\\先天性緑内障\\データ引継ぎ\\children_d\\2761_10.jpg', 'F:\\先天性緑内障\\データ引継ぎ\\children_d\\2761_2.jpg', 'F:\\先天性緑内障\\データ引継ぎ\\children_d\\2761_3.jpg', 'F:\\先天性緑内障\\データ引継ぎ\\children_d\\2761_9.jpg', 'F:\\先天性緑内障\\データ引継ぎ\\children_d\\3422_1.jpg', 'F:\\先天性緑内障\\データ引継ぎ\\children_d\\3422_2.jpg', 'F:\\先天性緑内障\\データ引継ぎ\\children_d\\4377_19.jpg', 'F:\\先天性緑内障\\データ引継ぎ\\children_d\\4377_20.jpg', 'F:\\先天性緑内障\\データ引継ぎ\\children_d\\4407_3.jpg', 'F:\\先天性緑内障\\データ引継ぎ\\children_d\\4407_1.jpg', 'F:\\先天性緑内障\\データ引継ぎ\\children_d\\4407_2.jpg', 'F:\\先天性緑内障\\データ引継ぎ\\children_d\\4849_24.jpg', 'F:\\先天性緑内障\\データ引継ぎ\\chil

In [70]:
import codecs
import pandas as pd
import os
import numpy as np
import shutil
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import StratifiedGroupKFold
import pandas as pd
from PIL import Image
import tempfile
import time
import glob
import itertools

pd.set_option('display.max_rows', 500)

gla_ortho_path = r"F:\先天性緑内障\データ引継ぎ\children_d"
gla_eso_path = r"F:\先天性緑内障\データ引継ぎ\children_d__内斜視"
gla_exo_path = r"F:\先天性緑内障\データ引継ぎ\children_d__外斜視"
cont_ortho_path = r"F:\先天性緑内障\データ引継ぎ\children_control"
cont_eso_path = r"F:\先天性緑内障\データ引継ぎ\children_control__内斜視\内斜視かぶりなし"
cont_exo_path = r"F:\先天性緑内障\データ引継ぎ\children_control__外斜視\外斜視かぶりなし"


def make_path_list(dir):
    path_list =  [file for file in glob.glob(dir+"/*") if os.path.isfile(file) == True ]
    return path_list

def extract_class(path_list, className):
    class_list = list(itertools.repeat(className,len(path_list)))
    return class_list

def extract_ids(path_list):
    id_list = [os.path.basename(name).split("_")[0] for name in path_list]
    return(id_list)


gla_ortho_path_list = make_path_list(gla_ortho_path)
gla_eso_path_list = make_path_list(gla_eso_path)
gla_exo_path_list = make_path_list(gla_exo_path)
cont_ortho_path_list = make_path_list(cont_ortho_path)
cont_eso_path_list = make_path_list(cont_eso_path)
cont_exo_path_list = make_path_list(cont_exo_path)

#それぞれの項目（path, classes, ID）をリスト化
gla_dataset_path = gla_ortho_path_list + gla_eso_path_list + gla_exo_path_list
gla_classes = extract_class(gla_ortho_path_list, "ortho") + extract_class(gla_eso_path_list, "eso") + extract_class(gla_exo_path_list, "exo")
gla_id = extract_ids(gla_ortho_path_list) + extract_ids(gla_eso_path_list) + extract_ids(gla_exo_path_list)
cont_dataset_path = cont_ortho_path_list + cont_eso_path_list + cont_exo_path_list
cont_classes = extract_class(cont_ortho_path_list, "ortho") + extract_class(cont_eso_path_list, "eso") + extract_class(cont_exo_path_list, "exo")
cont_id = extract_ids(cont_ortho_path_list) + extract_ids(cont_eso_path_list) + extract_ids(cont_exo_path_list)

#convert to Numpy
gla_dataset_path = np.array(gla_dataset_path)
gla_classes = np.array(gla_classes)
gla_id = np.array(gla_id)
cont_dataset_path = np.array(cont_dataset_path)
cont_classes = np.array(cont_classes)
cont_id = np.array(cont_id)

print(len(gla_dataset_path))
print(len(cont_dataset_path))

204
634


In [ ]:
"""
------test_dataset[0]
  |
  |---train_dataset_gla[0]----0
  |                        |--1
  |                        |--2
  |                        |--3
  |                        |--4
  |---train_dataset_cont[0]----0
  |                         |--1
  |                         |--2
  |                         |--3
  |                         |--4
  |---val_dataset_gla[0]----0
  |                      |--1
  |                      |--2
  |                      |--3
  |                      |--4
  |---val_dataset_cont[0]----0
  |                       |--1
  |                       |--2
  |                       |--3
  |                       |--4
  |---test_dataset[1]
  ...

"""

In [ ]:
train_dataset_gla, val_dataset_gla,train_dataset_cont, val_dataset_cont, test_dataset = [], [], [], [], []

#まずglaのデータセットから1人分を抜き出す（LeaveOneGroupOut)
# one group leave out 見本
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneGroupOut.html#sklearn.model_selection.LeaveOneGroupOut
# 今回のケースでは、groupがIDに該当
logo = LeaveOneGroupOut()
logo.get_n_splits(gla_dataset_path, gla_classes, gla_id)
logo.get_n_splits(groups=gla_id)  # 'groups' is always required

k=0
for remain_index, test_index in logo.split(gla_dataset_path, gla_classes, gla_id):
    gla_dataset_path_remain, gla_dataset_path_test = gla_dataset_path[remain_index], gla_dataset_path[test_index]
    gla_classes_remain, gla_classes_test = gla_classes[remain_index], gla_classes[test_index]
    gla_id_remain, gla_id_test = gla_id[remain_index], gla_id[test_index]
    #print(gla_dataset_path, gla_dataset_path_test, gla_id_train, gla_id_test)
    #print("test: "+gla_id_test[0])
    #print("TRAIN:", remain_index, "TEST:", test_index)
    #print(gla_dataset_path_test[0])
    test_dataset.append(gla_dataset_path_test.tolist())

    #抜き出した残りのglaについてStratified group 5-foldをかける
    # example of stratified group Kfold　見本
    # 今回のケースでは、groupがID、yがclassesに該当

    cv = StratifiedGroupKFold(n_splits=5)
    m=0 
    train_miniset, val_miniset =  [0 for i in range(0, 5)], [0 for i in range(0, 5)]
    for train_idxs, val_idxs in cv.split(gla_dataset_path_remain, gla_classes_remain, gla_id_remain):
        #print("TRAIN:", gla_classes_remain[train_idxs])
        #print("      ", gla_id_remain[train_idxs])
        #print("      ", gla_dataset_path_remain[train_idxs])
        #print(" TEST:", gla_classes_remain[val_idxs])
        #print("      ", gla_id_remain[val_idxs])
        #print("      ", gla_dataset_path_remain[val_idxs])
        train_miniset[m] = gla_dataset_path_remain[train_idxs].tolist()
        val_miniset[m] = gla_dataset_path_remain[val_idxs].tolist()
        m+=1
    train_dataset_gla.append(train_miniset)
    val_dataset_gla.append(val_miniset)
    #print("train_dataset_added label[gla] " + str(k))
    k+=1

    #controlについてStratified group 5-foldをかける
    m=0 
    train_miniset, val_miniset =  [0 for i in range(0, 5)], [0 for i in range(0, 5)]
    for train_idxs, val_idxs in cv.split(cont_dataset_path, cont_classes, cont_id):
        #print("TRAIN:", cont_classes[train_idxs])
        #print("      ", cont_id[train_idxs])
        #print("      ", cont_dataset_path[train_idxs])
        #print(" TEST:", cont_classes[val_idxs])
        #print("      ", cont_id[val_idxs])
        #print("      ", cont_dataset_path[val_idxs])
        train_miniset[m] = cont_dataset_path[train_idxs].tolist()
        val_miniset[m] = cont_dataset_path[val_idxs].tolist()
        m+=1
    train_dataset_cont.append(train_miniset)
    val_dataset_cont.append(val_miniset)

        
#print(len(train_dataset_gla))    
#print(len(val_dataset_gla))
#print(val_dataset_gla)
#print(len(train_dataset_cont))    
#print(len(val_dataset_cont))
#print(len(test_dataset))


#同じくcontのデータセットから1人分抜き出してLeaveOneGroupOutをする
logo = LeaveOneGroupOut()
logo.get_n_splits(cont_dataset_path, cont_classes, cont_id)
logo.get_n_splits(groups=cont_id)  # 'groups' is always required

k=0
for remain_index, test_index in logo.split(cont_dataset_path, cont_classes, cont_id):
    cont_dataset_path_remain, cont_dataset_path_test = cont_dataset_path[remain_index], cont_dataset_path[test_index]
    cont_classes_remain, cont_classes_test = cont_classes[remain_index], cont_classes[test_index]
    cont_id_remain, cont_id_test = cont_id[remain_index], cont_id[test_index]
    #print(cont_dataset_path_test[0])
    test_dataset.append(cont_dataset_path_test.tolist())

    #抜き出した残りのcontについてStratified group 5-foldをかける

    cv = StratifiedGroupKFold(n_splits=5)
    m=0 
    train_miniset, val_miniset =  [0 for i in range(0, 5)], [0 for i in range(0, 5)]
    for train_idxs, val_idxs in cv.split(cont_dataset_path_remain, cont_classes_remain, cont_id_remain):
        train_miniset[m] = cont_dataset_path_remain[train_idxs].tolist()
        val_miniset[m] = cont_dataset_path_remain[val_idxs].tolist()
        m+=1
    train_dataset_cont.append(train_miniset)
    val_dataset_cont.append(val_miniset)

    #glaについてStratified group 5-foldをかける
    m=0 
    train_miniset, val_miniset =  [0 for i in range(0, 5)], [0 for i in range(0, 5)]
    for train_idxs, val_idxs in cv.split(gla_dataset_path, gla_classes, gla_id):
        train_miniset[m] = gla_dataset_path[train_idxs].tolist()
        val_miniset[m] = gla_dataset_path[val_idxs].tolist()
        m+=1
    train_dataset_gla.append(train_miniset)
    val_dataset_gla.append(val_miniset)
    #print("train_dataset_added label[cont] "+ str(k))
    k+=1
        
print(len(train_dataset_gla))    
print(len(val_dataset_gla))
print(len(train_dataset_cont))    
print(len(val_dataset_cont))
print(len(test_dataset))


In [88]:

train_dataset = train_dataset_gla[0][0] + train_dataset_cont[0][0]
train_dataset_label = list(itertools.repeat(1, len(train_dataset_gla[0][0])))+list(itertools.repeat(1, len(train_dataset_cont[0][0])))
len(train_dataset_label)


661

In [91]:
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

class SimpleImageDataset(Dataset):
    def __init__(self, img_list, label_list, transform):
        self.transform = transform
        self.img_list = img_list
        self.label_list = label_list
        self.item_dict = {}
        self.age = []


    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        img = self.img_list[idx]
        pilr_image = Image.open(image_path).convert("RGB")
        tensor_image = self.transform(pilr_image)
        target = torch.tensor(label_list[idx])      
        return tensor_image, target

ModuleNotFoundError: ignored

In [94]:
!pip3 install torch==1.10.1+cu102 torchvision==0.11.2+cu102 torchaudio===0.10.1+cu102 -f https://download.pytorch.org/whl/cu102/torch_stable.html


Looking in links: https://download.pytorch.org/whl/cu102/torch_stable.html


ERROR: Could not find a version that satisfies the requirement torch==1.10.1+cu102 (from versions: none)
ERROR: No matching distribution found for torch==1.10.1+cu102


In [ ]:
train_dataset = SimpleImageDataset(os.path.join(DATASET_PATH, TRAIN_FOLDER_NAME), os.path.join(DATASET_PATH, FILENAME_LABELCSV), train_data_transforms)


In [1]:
import torch

ModuleNotFoundError: ignored